## Passive 전략의 포트폴리오를 자동 관리해주는 프로그램
---
현재 포트폴리오를 체크 -> 투자원칙에 따른 비중 계산 -> 비중에 맞게 자동 매매 진행

### 초기 세팅

#### 초기 투자자금 결정 - MDD,Cash Flow기반 전략

방법 :
    1. 전체 포트폴리오 최대 손실금액(PORT_BALANCE*PORT_MDD) = 1 month Cash Flow가 되도록 PORT_BALANCE 결정      
    즉 PORT_BALANCE = CF/PORT_MDD  
    2. 더 안전한 투자를 위해PORT_BALANCE에 Safe_Guard 비율 곱하기

In [3]:
PORT_MDD = 0.2

CF = 1000000

SAFE_GUARD = 0.5 
PORT_BALANCE = SAFE_GUARD*(CF/PORT_MDD)
PORT_BALANCE

2500000.0

#### 매달 불입금 결정 - 고정금액 불입
    1. 불입할 금액 액수를 임의로 정한다
    2. 최대손실금액((PORT_BALANCE+ADDON_BALANCE)*PORT_MDD)를 새로이 계산해서 타당한 금액인지 아닌지 고려한다
    3. 타당할 경우 그대로 불입한다

In [6]:
ADDON_BALANCE = 200000 
print("최대손실금액 :",(PORT_BALANCE+ADDON_BALANCE) * PORT_MDD)
#감당가능할 경우 불입 가능

최대손실금액 : 540000.0


### 전략 자동 리밸런싱 

전략명 : 

    주식형 포트폴리오 with 모멘텀 스코어 리밸런싱 (long only)

리밸런싱 지표 : 

    RSI/스토캐스틱 지표에 따른 스코어 산정, 스코어에 따른 순위 계산
    ->RSI가 낮으면 최근 하락세를 의미하므로 비중축소
    ->스토캐스틱이 낮으면 저점 근처라는 말이므로 역시 비중축소
    ->RSI,스토캐스틱의 평균을 낸 후, 평균을 스코어로 설정

리밸런싱 기간 :
    2주(10거래일)
    
방식 :
    1. 이전 투자기간의 성과 손익 기록
    2. 스코어 계산 및 순위에 따른 가중치 계산
    3. 신규 불입금 입금 및 신규 불입금을 고려한 신규 자산총액 계산
    4. 신규 자산총액을 기준으로, 계산된 가중치로부터 리밸런싱을 위한 거래총액 계산
    5. 거래총액에 맞게 시장가 거래 (정확하게 내가 원하는 비중/가격으로 맞출순 없고, 슬리피지 감수해야됨)
    
원칙 :
    1. 리밸런싱으로 얻은 이익은 반드시 전부 현금화한다. 이익을 재투자하지 않고 자산총액의 증가와 분리한다.
    2. 리밸런싱으로 얻은 손해도 반드시 전부 벌충한다. 손실을 자산총액의 감소와 분리한다.
    3. 이를 통해 불입금으로 인한 이익/손해의 희석을 방지하고 정확하게 포트폴리오의 성과를 측정한다

In [7]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr 
import win32com.client
from datetime import datetime,timedelta
import ctypes

In [8]:
#작업경로를 프로젝트 최상위폴더로 설정
import os

os.chdir('..')

#### 크레온 셋업 및 기존 잔고 계산

In [8]:
# 크레온 관리자권한, 로그인, 주문초기화 확인
def check_connection() :
    #콘솔 관리자권한 실행 확인
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        
    #크레온 로그인 확인
    instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    if instCpCybos.IsConnect == 0 :
        print("PLUS가 정상적으로 연결되지 않음")
    else :
        print("PLUS가 정상적으로 연결됨")
        
    #로그인 및 주문 초기화
    instCpTdUtil = win32com.client.Dispatch("CpTrade.CpTdUtil")
    instCpTd0311 = win32com.client.Dispatch("CpTrade.CpTd0311")

    if instCpTdUtil.TradeInit() != 0 :
        print("주문 초기화 실패")
    else :
        print("주문 초기화 성공")

In [9]:
check_connection()

정상: 관리자권한으로 실행된 프로세스입니다.
PLUS가 정상적으로 연결됨
주문 초기화 성공


In [9]:
#이전기간 손익 결과 계산
#타임스탬프는 기간 종료 후 다음 리밸런싱일 기준으로 찍힘
def calc_result(result_df,begin=False) :
    if begin :
        open_balance = 2000000
        close_balance = 2000000
        next_balance = 2000000
        date = datetime.strptime("2020-03-23","%Y-%m-%d")
        
    else :
        objTrade =  win32com.client.Dispatch("CpTrade.CpTdUtil")
        initCheck = objTrade.TradeInit(0)
        if (initCheck != 0):
            print("주문 초기화 실패")
        
        #현재 계좌잔고 받아오기
        instCpTdUtil = win32com.client.Dispatch("CpTrade.CpTdUtil")
        acc = instCpTdUtil.AccountNumber[0]
        accFlag = instCpTdUtil.GoodsList(acc, -1)  # 주식상품 구분
        objRq = win32com.client.Dispatch("CpTrade.CpTd6033")
        objRq.SetInputValue(0, acc)  # 계좌번호
        objRq.SetInputValue(1, accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        objRq.SetInputValue(2, 50)  #  요청 건수(최대 50개)

        objRq.BlockRequest()

        cnt = objRq.GetHeaderValue(7)

        close_balance = 0
        #포트폴리오 내에 포함되는 주식들만 선택
        print("종목코드 종목명 신용구분 체결잔고수량 체결장부단가 평가금액 평가손익")
        for i in range(cnt): 
            code = objRq.GetDataValue(12, i)  # 종목코드

            evalValue = objRq.GetDataValue(9, i) # 평가금액(천원미만은 절사 됨)

            if code[1:] in assets.values() :
                close_balance += evalValue
        
        close_balance += objRq.GetHeaderValue(9)
        open_balance = result_df.iloc[-1,2]
        next_balance = open_balance + 200000
        date = result_df.index[-1]+timedelta(14)
        
    result = pd.Series()
    result.name = date #리밸런싱 당일
    result['OpenBalance'] = open_balance
    result['CloseBalance'] = close_balance
    result['NextBalance'] = next_balance
    result['Income'] = close_balance-open_balance
    result['Change'] = (close_balance-open_balance)/open_balance
    
    result_df = result_df.append(result)
    result_df = result_df[['OpenBalance','CloseBalance','NextBalance','Income','Change']]
    return result_df

In [ ]:
#포트폴리오 히스토리파일 초기화
port_history = pd.DataFrame()
port_history = calc_result(port_history,True)
print(port_history)
port_history.to_csv('history/port_history.csv')

In [ ]:
#포트폴리오 히스토리 손익 갱신
port_history = pd.read_csv('history/port_history.csv')
port_history = port_history.rename(columns= {'Unnamed: 0':'Date'})
port_history['Date']= pd.to_datetime(port_history['Date'])
port_history = port_history.set_index('Date')
port_history = calc_result(port_history)
print(port_history)
port_history.to_csv('history/port_history.csv')
#port_history.to_csv('port_history/port_history.csv')

In [35]:
#다음기간 자산총액 재설정
PORT_BALANCE = port_history.iloc[-1,2]
PORT_BALANCE

2200000.0

#### 리밸런싱 비중 계산

In [13]:
#포트폴리오 포함 자산군 정의
assets = {'KOSPI200':'069500',
          'S&P':'219480','미국채30':'304660','달러레버':'261250',
          'TOPIX레버':'196030','엔레버':'292570',
          'CSI300':'192090','STOXX':'225050',
          'MSCI리츠':'182480',
          '원유레버':'530031','골드':'132030'
         }
assets_ticker = {'069500':'KOSPI200',
          '219480':'S&P','304660':'미국채30','261250':'달러레버',
          '196030':'TOPIX레버','292570':'엔레버',
          '192090':'CSI300','225050':'STOXX',
          '182480':'MSCI리츠',
          '530031':'원유레버','132030':'골드'
         }

port_data = dict()
for key,value in assets.items() :
    port_data[key] = fdr.DataReader(value, '1997')

In [14]:
#포트폴리오 초기 자산비중 임의 설정
base_weight = pd.Series({'KOSPI200':0.2,
          'S&P':0.2,'미국채30':0,'달러레버':0.1,
          'TOPIX레버':0,'엔레버':0.1,
          'CSI300':0,'STOXX':0,
          'MSCI리츠':0.2,
          '원유레버':0.1,'골드':0.1
         })

mask_weight = pd.Series({'KOSPI200':1,
          'S&P':1,'미국채30':0,'달러레버':1,
          'TOPIX레버':0,'엔레버':1,
          'CSI300':0,'STOXX':0,
          'MSCI리츠':1,
          '원유레버':1,'골드':1
         })

base_balance = pd.Series({'KOSPI200':0,
          'S&P':0,'미국채30':0,'달러레버':0,
          'TOPIX레버':0,'엔레버':0,
          'CSI300':0,'STOXX':0,
          'MSCI리츠':0,
          '원유레버':0,'골드':0
         })

base_shares = pd.Series({'KOSPI200':0,
          'S&P':0,'미국채30':0,'달러레버':0,
          'TOPIX레버':0,'엔레버':0,
          'CSI300':0,'STOXX':0,
          'MSCI리츠':0,
          '원유레버':0,'골드':0
         })

In [17]:
#새 포트폴리오 지표 및 리밸런스 weight 계산함수
def resample_data(asset,period) : 
    rawdata = port_data[asset]
    resampledata = rawdata.resample(period)
    newdata = pd.DataFrame()
    newdata['Open'] = resampledata.first()['Open']
    newdata['High'] = resampledata.max()['High']
    newdata['Low'] = resampledata.min()['Low']
    newdata['Close'] = resampledata.last()['Close']
    newdata['Volume'] = resampledata.sum()['Volume']
    newdata['Change'] = (newdata['Close'] -newdata['Open'])/newdata['Open']
    
    return newdata
    
def calc_rsi(asset,day) :
    df = port_data[asset]
    df['Diff_close_abs'] = (df['Close'] - df['Close'].shift(1)).abs()
    df['Diff_close_pos'] = df['Close'] - df['Close'].shift(1)
    df['Diff_close_pos'] = df[df['Diff_close_pos']>0]['Diff_close_pos']
    df['Diff_close_pos'] = df['Diff_close_pos'].fillna(0)
    df['RSI'] = df.rolling(day).sum()['Diff_close_pos'] /df.rolling(day).sum()['Diff_close_abs']

    return df.RSI[-1]

def calc_stoch(asset,day) :
    df = port_data[asset]
    df['Min'] = df.rolling(day).min()['Close']
    df['Max'] = df.rolling(day).max()['Close']
    df['Stoch'] = (df['Close']-df['Min'])/(df['Max']-df['Min'])

    return df.Stoch[-1]


def calc_weight() :
    scores = pd.Series(dtype="float64")
    weights = pd.Series(dtype="float64")
    
    for key in assets.keys() :
        score = (calc_rsi(key,10) + calc_stoch(key,10))/2
        scores[key] = score
    
    scores = scores*mask_weight
    score_total = scores.sum()
    
    for key in assets.keys() :
        weight = scores[key]/score_total
        weights[key] = weight
    
    return weights

In [24]:
#리밸런싱된 새 자산별 웨이트
new_asset_weight = calc_weight()
print("제대로 계산되었는지 확인(웨이트 총 합이 1에 가까워야 함 :")
print("총 합 :", new_asset_weight.sum())
new_asset_weight

제대로 계산되었는지 확인(웨이트 총 합이 1에 가까워야 함 :
총 합 : 0.9999999999999999


KOSPI200    0.169824
S&P         0.169806
미국채30       0.000000
달러레버        0.105741
TOPIX레버     0.000000
엔레버         0.084895
CSI300      0.000000
STOXX       0.000000
MSCI리츠      0.159201
원유레버        0.168314
골드          0.142219
dtype: float64

In [26]:
#리밸런싱결과 weight로부터 새 자산별 잔고 계산
print("이번기간 자산 총액은 :",PORT_BALANCE)
new_asset_balance = (new_asset_weight*PORT_BALANCE)
print("제대로 계산되었는지 확인(자산별 잔고 합이 자산총액에 가까워야 함 :")
print("총 합 :", new_asset_balance.sum())
new_asset_balance

이번기간 자산 총액은 : 2500000.0
제대로 계산되었는지 확인(자산별 잔고 합이 자산총액에 가까워야 함 :
총 합 : 2500000.0


KOSPI200    424560.526730
S&P         424515.619781
미국채30            0.000000
달러레버        264352.441496
TOPIX레버          0.000000
엔레버         212237.270532
CSI300           0.000000
STOXX            0.000000
MSCI리츠      398002.935610
원유레버        420784.596956
골드          355546.608895
dtype: float64

In [ ]:
#현재 보유 포트폴리오 자산별 잔고 계산함수
def calc_asset_balance() :
    asset_balance = base_balance #동일객체이므로 in-place임에 주의

    instCpTdUtil = win32com.client.Dispatch("CpTrade.CpTdUtil")
    acc = instCpTdUtil.AccountNumber[0]
    accFlag = instCpTdUtil.GoodsList(acc, 1)  # 주식상품 구분
    objRq = win32com.client.Dispatch("CpTrade.CpTd6033")
    objRq.SetInputValue(0, acc)  # 계좌번호
    objRq.SetInputValue(1, accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
    objRq.SetInputValue(2, 50)  #  요청 건수(최대 50)

    objRq.BlockRequest()
    cnt = objRq.GetHeaderValue(7)

    #포트폴리오 내에 포함되는 주식들만 선택
    print("종목코드 종목명 신용구분 체결잔고수량 체결장부단가 평가금액 평가손익")
    for i in range(cnt): 
        code = objRq.GetDataValue(12, i)  # 종목코드
        name = objRq.GetDataValue(0, i)  # 종목명

        cashFlag = objRq.GetDataValue(1, i)  # 신용구분
        date = objRq.GetDataValue(2, i)  # 대출일
        amount = objRq.GetDataValue(7, i) # 체결잔고수량
        buyPrice = objRq.GetDataValue(17, i) # 체결장부단가
        evalValue = objRq.GetDataValue(9, i) # 평가금액(천원미만은 절사 됨)
        evalPerc = objRq.GetDataValue(11, i) # 평가손익

        if code[1:] in assets.values() :
            print(code, name, cashFlag, amount, buyPrice, evalValue, evalPerc)
            asset_balance[assets_ticker[code[1:]]] = evalValue
    return asset_balance

In [ ]:
asset_balance = calc_asset_balance()
asset_balance

In [ ]:
#리밸런싱된 목표잔고와 현재잔고의 차액 계산
trading_balance = new_asset_balance - asset_balance
trading_balance

In [ ]:
#자산 실시간 예상체결가 획득
def calc_asset_price() :
    asset_price = pd.Series(dtype='int64')

    objStockMst = win32com.client.Dispatch("DsCbo1.StockMst")
    for key,value in assets.items() :
        if key == '원유레버' : #원유레버와 같은 ETN은 종목코드 Q로 시작함
            objStockMst.SetInputValue(0, 'Q'+value)  
        else : objStockMst.SetInputValue(0, 'A'+value)   #종목 코드

        objStockMst.BlockRequest()

        rqStatus = objStockMst.GetDibStatus()
        rqRet = objStockMst.GetDibMsg1()
        #print("통신상태", rqStatus, rqRet)

        exFlag = objStockMst.GetHeaderValue(58) #예상체결가 구분 플래그
        
        #exPrice = objStockMst.GetHeaderValue(55) #예상체결가 - 현재가와 괴리 큼
        Price = objStockMst.GetHeaderValue(11)
        asset_price[key] = Price
    return asset_price

In [ ]:
asset_price = calc_asset_price()
asset_price

In [ ]:
#예상체결가로부터 최종 리밸런싱 거래주식수 획득
trading_shares = (trading_balance/asset_price).apply(int)
trading_shares

#### 매수매도 주문부

In [ ]:
"""
호가에 대한 이해
    현재가를 중심으로 매도는 아래에서부터 1호가,2호가..
    현재가를 중심으로 매수는 위에서부터 1호가, 2호가

해야되는 일
     목표잔고(이후 fix) -> (루프 시작점)현재잔고 -> 차액계산 -> 현재가 수신 -> 목표 거래개수 계산 -> 현재가와 목표개수만큼 주문
    ->각 자산 주문체결 확인 -> 체결시 해당자산은 체결체크 -> 미체결시 기존주문 취소 및 루프 시작점 회귀
"""

In [42]:
#잔고값을 받아 매수,매도주문을 내는 함수
def place_order(code,price,shares) :
    g_objCpTrade = win32com.client.Dispatch("CpTrade.CpTdUtil")
    acc = g_objCpTrade.AccountNumber[0]  # 계좌번호
    accFlag = g_objCpTrade.GoodsList(acc, 1)  # 주식상품 구분

    objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
    if shares > 0 :
        objStockOrder.SetInputValue(0, "2") #매수
    elif shares < 0 :
        objStockOrder.SetInputValue(0, "1") #매도
        shares *= -1
    objStockOrder.SetInputValue(1, acc )   #  계좌번호
    objStockOrder.SetInputValue(2, accFlag[0])   # 상품구분 - 주식 상품 중 첫번째
    objStockOrder.SetInputValue(3, code)   # 종목코드 - A003540 - 대신증권 종목
    objStockOrder.SetInputValue(4, shares)   # 매수수량 10주
    objStockOrder.SetInputValue(5, price)   # 주문단가  - 14,100원
    objStockOrder.SetInputValue(7, "0")   # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
    objStockOrder.SetInputValue(8, "01")   # 주문호가 구분코드 - 01: 보통

    # 매수 주문 요청
    objStockOrder.BlockRequest()

In [43]:
g_objCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
g_objCpStatus = win32com.client.Dispatch("CpUtil.CpCybos")
g_objCpTrade = win32com.client.Dispatch("CpTrade.CpTdUtil")

class orderData:
    def __init__(self):
        self.code = ""          # 종목코드
        self.name = ""          # 종목명
        self.orderNum = 0       # 주문번호
        self.orderPrev = 0      # 원주문번호
        self.orderDesc = ""     # 주문구분내용
        self.amount = 0     # 주문수량
        self.price = 0      # 주문 단가
        self.ContAmount = 0  # 체결수량
        self.credit = ""     # 신용 구분 "현금" "유통융자" "자기융자" "유통대주" "자기대주"
        self.modAvali = 0  # 정정/취소 가능 수량
        self.buysell = ""  # 매매구분 코드  1 매도 2 매수
        self.creditdate = ""    # 대출일
        self.orderFlag = ""     # 주문호가 구분코드
        self.orderFlagDesc = "" # 주문호가 구분 코드 내용
 
        # 데이터 변환용
        self.concdic = {"1": "체결", "2": "확인", "3": "거부", "4": "접수"}
        self.buyselldic = {"1": "매도", "2": "매수"}
 
    def debugPrint(self):
        print("%s, %s, 주문번호 %d, 원주문 %d, %s, 주문수량 %d, 주문단가 %d, 체결수량 %d, %s, "
              "정정가능수량 %d, 매수매도: %s, 대출일 %s, 주문호가구분 %s %s"
              %(self.code, self.name, self.orderNum, self.orderPrev, self.orderDesc, self.amount, self.price,
                self.ContAmount,self.credit,self.modAvali, self.buyselldic.get(self.buysell),
                self.creditdate,self.orderFlag, self.orderFlagDesc))

def do_rebalance_once() : #수동으로 한번 리밸런싱 수행하는 함수
    #미체결주문 정보 획득
    dicOrderList= dict()
    
    objRq = win32com.client.Dispatch("CpTrade.CpTd5339")
    acc = g_objCpTrade.AccountNumber[0]  # 계좌번호
    accFlag = g_objCpTrade.GoodsList(acc, 1)  # 주식상품 구분
    
    objRq.SetInputValue(0, acc)
    objRq.SetInputValue(1, accFlag[0])
    objRq.SetInputValue(4, "0") # 전체
    objRq.SetInputValue(5, "1") # 정렬 기준 - 역순
    objRq.SetInputValue(6, "0") # 전체
    objRq.SetInputValue(7, 20) # 요청 개수 - 최대 20개
    
    print("[Cp5339] 미체결 데이터 조회 시작")
    # 미체결 연속 조회를 위해 while 문 사용
    while True :
        ret = objRq.BlockRequest()
        if objRq.GetDibStatus() != 0:
            print("통신상태", objRq.GetDibStatus(), objRq.GetDibMsg1())
            return False

        if (ret == 2 or ret == 3):
            print("통신 오류", ret)
            return False;

        # 통신 초과 요청 방지에 의한 요류 인 경우
        while (ret == 4) : # 연속 주문 오류 임. 이 경우는 남은 시간동안 반드시 대기해야 함.
            remainTime = g_objCpStatus.LimitRequestRemainTime
            print("연속 통신 초과에 의해 재 통신처리 : ",remainTime/1000, "초 대기" )
            time.sleep(remainTime / 1000)
            ret = objRq.BlockRequest()


        # 수신 개수
        cnt = objRq.GetHeaderValue(5)
        print("[Cp5339] 수신 개수 ", cnt)
        if cnt == 0 :
            break

        for i in range(cnt):
            item = orderData()
            item.orderNum = objRq.GetDataValue(1, i)
            item.orderPrev  = objRq.GetDataValue(2, i)
            item.code  = objRq.GetDataValue(3, i)  # 종목코드
            item.name  = objRq.GetDataValue(4, i)  # 종목명
            item.orderDesc  = objRq.GetDataValue(5, i)  # 주문구분내용
            item.amount  = objRq.GetDataValue(6, i)  # 주문수량
            item.price  = objRq.GetDataValue(7, i)  # 주문단가
            item.ContAmount = objRq.GetDataValue(8, i)  # 체결수량
            item.credit  = objRq.GetDataValue(9, i)  # 신용구분
            item.modAvali  = objRq.GetDataValue(11, i)  # 정정취소 가능수량
            item.buysell  = objRq.GetDataValue(13, i)  # 매매구분코드
            item.creditdate  = objRq.GetDataValue(17, i)  # 대출일
            item.orderFlagDesc  = objRq.GetDataValue(19, i)  # 주문호가구분코드내용
            item.orderFlag  = objRq.GetDataValue(21, i)  # 주문호가구분코드

            # 사전과 배열에 미체결 item 을 추가
            dicOrderList[item.orderNum] = item

        # 연속 처리 체크 - 다음 데이터가 없으면 중지
        if objRq.Continue == False :
            print("[Cp5339] 연속 조회 여부: 다음 데이터가 없음")
            break
    #기존 미체결주문 싹다 취소
    
    objCancelOrder = win32com.client.Dispatch("CpTrade.CpTd0314")
    for key,value in dicOrderList.items() : 
        objCancelOrder.SetInputValue(1, value.orderNum)  # 원주문 번호 - 정정을 하려는 주문 번호
        objCancelOrder.SetInputValue(2, acc)  # 상품구분 - 주식 상품 중 첫번째
        objCancelOrder.SetInputValue(3, accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        objCancelOrder.SetInputValue(4, value.code)  # 종목코드
        objCancelOrder.SetInputValue(5,0)  # 정정 수량, 0 이면 잔량 취소임
        objCancelOrder.BlockRequest()

    #현재잔고 획득
    asset_balance = calc_asset_balance()
    #현재가 획득
    trading_balance = new_asset_balance - asset_balance
    asset_price = calc_asset_price()
    #트레이딩개수 획득
    trading_shares = (trading_balance/asset_price).apply(int)
    #트레이딩개수와 현재가로 주문내기 - 개수가 양수면 매수주문, 개수가 음수면 매도주문
    for key,value in assets.items() :
        if key == '원유레버' : code = 'Q'+value
        else : code = 'A'+value
        
        if trading_shares[key] == 0 : continue
        else : place_order(code,asset_price[key],trading_shares[key]) 

In [ ]:
#수동으로 주문 실행
#개선점 : 이거 주문 제대로 잘 들어갔는지 response 확인 필요
#원유/금 주문이 제대로 안될 경우 -> ETN 거래신청 안해서 그럼
do_rebalance_once()